# Installation et imports

### Installation des modules 

In [418]:
%pip install Pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [419]:
%pip install Matplotlib 


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [420]:
%pip install Scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [421]:
%pip install Numpy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import et données

In [422]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import math


In [505]:
data = pd.read_csv("../dfClean.csv")

counts = data['descr_grav'].value_counts()

total_0 = counts[0]/len(data)
total_1 = counts[1]/len(data)
total_2 = counts[2]/len(data)
total_3 = counts[3]/len(data)

Proportion_0 = data[data['descr_grav'] == 0].sample(n=int(7500*total_0))
Proportion_1 = data[data['descr_grav'] == 1].sample(n=int(7500*total_1))
Proportion_2 = data[data['descr_grav'] == 2].sample(n=int(7500*total_2))
Proportion_3 = data[data['descr_grav'] == 3].sample(n=int(7500*total_3))

data_reduit = pd.concat([Proportion_0,Proportion_1, Proportion_2, Proportion_3])
data_reduit = data_reduit.reset_index(drop=True)
print(len(data_reduit))
display(data_reduit)

7498


,date,latitude,longitude,descr_cat_veh,descr_agglo,place,descr_dispo_secu,descr_grav
0,22519,48.8412,2.300200,0,0,0,14,0
1,17376,48.8722,2.312400,4,0,0,7,0
2,2483,45.7167,4.800000,0,0,0,14,0
3,27562,45.7500,-0.633333,0,0,3,14,0
4,23320,47.8167,7.083330,0,0,0,14,0
...,...,...,...,...,...,...,...,...
7493,21581,45.3000,-0.933333,2,1,0,11,3
7494,23368,49.5667,3.050000,0,1,0,14,3
7495,8665,46.0833,5.866670,0,1,0,14,3
7496,9624,48.8667,2.716670,2,0,0,11,3


# Répartition des données

### Répartition holdout 

In [424]:
from sklearn.model_selection import train_test_split

In [507]:
def holdout(dataframe, colonne_predict, num_rep, train_s, test_s ):
        liste_data_train_test = []
        liste_data = []
        clean_data = dataframe.drop(colonne_predict, axis = 1)
        y = dataframe[colonne_predict]
        for i in range(num_rep):
                X_train, X_test, y_train, y_test = train_test_split(clean_data,y, train_size= train_s, test_size = test_s)
                X_train = X_train.reset_index(drop=True)
                y_train = y_train.reset_index(drop=True)
                X_test = X_test.reset_index(drop=True)
                y_test = y_test.reset_index(drop=True)
                liste_data_train_test.append([X_train, X_test, y_train, y_test])
        return liste_data_train_test

### Répartition Leave One Out

In [427]:
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

In [506]:
def leaveoneout(dataframe, colonne_predict,k):
    liste_y_pred = []
    liste_y_test = []
    X = data.drop(colonne_predict, axis = 1) 
    y = data[colonne_predict]
    LOO = LeaveOneOut()
    knn = KNeighborsClassifier(n_neighbors=k)
    
    for train_index, test_index in (LOO.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_y_pred.append(y_pred)
        liste_y_test.append(y_test)
        
    return liste_y_pred, liste_y_test

# Classification avec KNN

### Fonctions from scratch

In [467]:
#Fonction de classification knn
def KnnClassification_distance_euclidienne(ligne,y_train,df,k):
    dictList = [None] * len(df)
    liste_y_pred = []
    for index, row in df.iterrows():
        dictList[index] = np.linalg.norm(ligne - row) #calcul de la distance euclidienne
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    for index,value in sorted_values:
        #firstk.append([df.loc[index]])
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

In [466]:
#Fonction de classification knn
def KnnClassification_distance_manhattan(ligne, y_train, df, k):
    dictList = [None] * len(df)
    liste_y_pred = []
    
    for index, row in df.iterrows():
        dictList[index] = np.sum(np.abs(ligne - row))  # Calcul de la distance de Manhattan
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    
    for index, value in sorted_values:
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

In [464]:
def KnnClassification_distance_sup_norme(ligne, y_train, df, k):
    dictList = [None] * len(df)
    liste_y_pred = []
    
    for index, row in df.iterrows():
        dictList[index] = np.max(np.abs(ligne - row))  # Calcul de la distance de la norme supérieure
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    
    for index, value in sorted_values:
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

### Scikit learn

In [443]:
#prediction holdout sklearn
def  KnnClassification_sk_learn(liste_data):
    knn = KNeighborsClassifier(n_neighbors=5)
    liste_y_pred_holdout_sk = []
    liste_y_test_holdout_sk = []
    for i in range(len(liste_data)):
        X_train = liste_data[i][0]
        X_test = liste_data[i][1]
        y_test = liste_data[i][3]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_y_pred_holdout_sk.append(y_pred)
        liste_y_test_holdout_sk.append(y_test)
    return liste_y_pred_holdout_sk, liste_y_test_holdout_sk

### Calcul de score

In [447]:
from sklearn.metrics import accuracy_score

In [508]:
#Répartition leave One Out
y_pred_LOO_5, y_test_LOO_5 = leaveoneout(data_reduit, "descr_grav", 5)
y_pred_LOO_7, y_test_LOO_7 = leaveoneout(data_reduit, "descr_grav", 7)
y_pred_LOO_10, y_test_LOO_10 = leaveoneout(data_reduit, "descr_grav", 10)

In [457]:
# Score répartition leave One Out
accuracy_LOO_5 = accuracy_score(y_test_LOO_5, y_pred_LOO_5)
accuracy_LOO_7 = accuracy_score(y_test_LOO_7, y_pred_LOO_7)
accuracy_LOO_10 = accuracy_score(y_test_LOO_10, y_pred_LOO_10)

print(accuracy_LOO_5*100)
print(accuracy_LOO_7*100)
print(accuracy_LOO_10*100)

41.714285714285715
39.75714285714286
40.41428571428571


In [471]:
#Répartition hold out
liste_data_holdout = []
#data_test = data.drop(columns=["latitude", "longitude"])
data_test = data_reduit
liste_data_holdout  = holdout(data_reduit,"descr_grav", 5, 0.8, 0.2)

# Prediction holdout from scratch
liste_y_pred_holdout_LOO_euclidienne = []
liste_y_pred_holdout_LOO_manhattan = []
liste_y_pred_holdout_LOO_sup_norme = []


X_test_0 = liste_data_holdout[0][1]
y_train_0  = liste_data_holdout[0][2]
X_train_0 = liste_data_holdout[0][0]
y_test_0 =  liste_data_holdout[0][3]

for i in range(len(X_test)):
    y_pred_holdout_sup_norme = KnnClassification_distance_sup_norme(X_test.loc[i], y_train, X_train, 5)
    liste_y_pred_holdout_LOO_sup_norme.append(y_pred_holdout_sup_norme)
    y_pred_holdout_euclidienne = KnnClassification_distance_euclidienne(X_test.loc[i], y_train, X_train, 5)
    liste_y_pred_holdout_LOO_euclidienne.append(y_pred_holdout_euclidienne)
    y_pred_holdout_manhattan = KnnClassification_distance_manhattan(X_test.loc[i], y_train, X_train, 5)
    liste_y_pred_holdout_LOO_manhattan.append(y_pred_holdout_manhattan)

for i in range(len(X_test)):
    y_pred_holdout_sup_norme = KnnClassification_distance_sup_norme(X_test.loc[i], y_train, X_train, 7)
    liste_y_pred_holdout_LOO_sup_norme.append(y_pred_holdout_sup_norme)
    y_pred_holdout_euclidienne = KnnClassification_distance_euclidienne(X_test.loc[i], y_train, X_train, 7)
    liste_y_pred_holdout_LOO_euclidienne.append(y_pred_holdout_euclidienne)
    y_pred_holdout_manhattan = KnnClassification_distance_manhattan(X_test.loc[i], y_train, X_train, 7)
    liste_y_pred_holdout_LOO_manhattan.append(y_pred_holdout_manhattan)

for i in range(len(X_test)):
    y_pred_holdout_sup_norme = KnnClassification_distance_sup_norme(X_test.loc[i], y_train, X_train, 10)
    liste_y_pred_holdout_LOO_sup_norme.append(y_pred_holdout_sup_norme)
    y_pred_holdout_euclidienne = KnnClassification_distance_euclidienne(X_test.loc[i], y_train, X_train, 10)
    liste_y_pred_holdout_LOO_euclidienne.append(y_pred_holdout_euclidienne)
    y_pred_holdout_manhattan = KnnClassification_distance_manhattan(X_test.loc[i], y_train, X_train, 10)
    liste_y_pred_holdout_LOO_manhattan.append(y_pred_holdout_manhattan)


KeyboardInterrupt: 

In [ ]:
# Score répartition leave One Out
accuracy_holdout_euclidienne = accuracy_score(y_test_0, liste_y_pred_holdout_LOO_euclidienne)
accuracy_holdout_manhattan = accuracy_score(y_test_0, liste_y_pred_holdout_LOO_manhattan)
accuracy_holdout_sup_norme = accuracy_score(y_test_0, liste_y_pred_holdout_LOO_sup_norme)

print("Score euclidienne:" accuracy_holdout_euclidienne*100)
print("Score manhattan:" accuracy_holdout_manhattan*100)
print("Score sup norme:" accuracy_holdout_sup_norme*100)

In [ ]:
#Score prediction holdout scikit learn
#for i in range(len(liste_y_pred_holdout_sk)):
liste_y_pred_holdout_sk, liste_y_test_holdout_sk = KnnClassification_sk_learn(liste_data_holdout)
accuracy_holdout_sk = accuracy_score(liste_y_test_holdout_sk, liste_y_pred_holdout_sk)